### Regression Models

이번 시간에서는 대표적인 Regression Model에 대한 개념들을 살펴보는 것으로 한다

#### 1. Bias - Variance Dilemma

다음 회귀모형을 고려해 보자

$$y_t = \beta_0 + \beta_1 x_t + \epsilon_t, ~~~~~ \epsilon_t \sim N(0,\sigma_\epsilon^2) $$

오차항은 다음과 같이 다시 쓸 수 있다

$$\epsilon_t = y_t - \hat{\beta_0} - \hat \beta_1 x_t$$

이를 제곱한 값을 Mean Squared Error라고 하는데, MSE는 다음과 같이 분해할 수 있다

$$\mathrm{MSE} = (E[\beta_0 + \beta_1 x_t - \hat \beta_0 - \hat \beta_1 x_t])^2 + \mathrm{Var}(\hat\beta_0 + \hat\beta_1 x_t) + \sigma_\epsilon^2$$

여기서 $E[\beta_0 + \beta_1 x_t - \hat \beta_0 - \hat \beta_1 x_t])^2$는 편향이라고 하고, $\mathrm{Var}(\hat\beta_0 + \hat\beta_1 x_t)$는 분산이라고 한다. $\sigma_\epsilon^2$은 줄일 수 없는 순수한 데이터의 잡음이다

즉, MSE optimization 과정에서 Variance와 Bias의 tradeoff 관계가 존재한다는 것을 유추할 수 있다. 이를 두고 계량경제학과 머신러닝 모형의 목적이 확연히 구분된다

- 계량경제학의 목적 : unbiased estimator중에서 Variance를 최소화하는 값을 추정한다 (BLUE, MVUE)
- 머신러닝의 목적 : **성능을 최대화하는** 지점에서 Bias와 Variance를 **적절히 조정한다** (minimize MSE)

#### 2. Support Vector Regression

1995년, Vapnik와 Cortes는 Support Vector Network라는 모형을 제안하였다

https://link.springer.com/article/10.1007/BF00994018

**2.1 Hard Margin Model**

Hard Margin Model의 손실함수는 다음을 최적화하는 것을 목적으로 한다

$$\mathrm{Loss_{SVR}} = \min_w ||w||^2 + \lambda(\frac{1}{2}\sum_{i = 1}^n (y_i - f(x_i))^2)$$

여기서 $||w||^2$는 데이터와 회귀선과의 거리를 나타내는데, 이는 선형회귀식에서 오차를 최소화하는 식과 유사함을 알 수 있다.

**2.2 Soft Margin Model**

<center><img src = "https://leejiyoon52.github.io/images/image_67.png" alt="My Image"></center>

과적합이나 기타 일반화 성능 향상을 위해, 보통은 아래와 같은 손실함수를 최적화하는 것을 목적으로 한다

$$\mathrm{Loss_{SVR}} = \min_w \frac{1}{2}||w||^2 + C\sum_{i = 1}^n (\xi_i + \xi_i^*)$$
$$\mathrm{such~that~} (w^T x_i + b) - y_i \leq \epsilon + \xi_i$$
$$y_i - (w^T x_i + b) \leq \epsilon + \xi_i^*$$
$$\xi_i, \xi_i^* \geq 0$$

추정된 회귀식에서 상단과 하단에 각각 $\epsilon$만큼 마진을 생성하여 추정식의 허용 범위를 넓혀 준다. 즉, margin 값을 크게 줄 수록 모형의 노이즈 허용 범위가 커지는 것이다

비용함수 최적화에 있어서 Lagrangian Problem으로 해를 찾는 것이 가능하지만, 그 과정이 길기 때문에 우선은 생략한다. Lagrangian Dual Problem은 다음과 같이 정리된다

$$\mathcal{L_D} = \frac{1}{2}\sum_{i,j = 1}^n (\alpha_i^* - \alpha_i)(\alpha_j^* - \alpha_j)\mathbf{x_i^Tx_j} - \epsilon \sum_{i,j = 1}^n(\alpha_i^* - \alpha_i) + \sum_{i,j = 1}^n y_i(\alpha_i^* - \alpha_i)$$
$$\mathrm{such ~ that ~} \sum_{i=1}^n (\alpha_i^* - \alpha_i) = 0, ~~~~~ \alpha_i, \alpha_i^* \in [0,C]$$

Lagrangian dual problem으로 재구성한 목적식은 $α$로 이루어져있는 convex하고, 연속적인 quadratic programming problem이다. 최적화 프로그램을 이용한다면 간편하게 $α$를 도출할 수 있다

**2.3 Kernel Function**

Support Vector Machine은 선형 모형이기 때문에 비선형 회귀식을 구해야 하는 경우가 상당히 제약적이었다. 이런 경우 Mapping function의 일종인 Kernel Function을 사용하여 해결하게 되었다

$$x = (x_1, x_2, \dots, x_p) \Rightarrow \phi(x) = z = (z_1, z_2, \dots, z_q)$$

<center><img src = "https://leejiyoon52.github.io/images/image_80.png" alt="My Image"></center>

즉, 중간에 비선형 공간을 추가하여 선형공간으로 다시 데이터를 평평하게 펴는 과정을 거치는 것이라 이해하면 좋다. Kernel Function이 포함된 Lagrangian Dual Problem은 다음과 같이 표현된다

$$\mathcal{L_D} = \frac{1}{2}\sum_{i,j = 1}^n (\alpha_i^* - \alpha_i)(\alpha_j^* - \alpha_j)\mathbf{K(x_i^,x_j)} - \epsilon \sum_{i,j = 1}^n(\alpha_i^* - \alpha_i) + \sum_{i,j = 1}^n y_i(\alpha_i^* - \alpha_i)$$

평면식과는 다르게 중간 과정에서 Kernel Function의 Mapping과정이 포함된것을 알 수 있다

#### 3. Tree Regression